In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
import os 
from dotenv import load_dotenv
from time import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
#loader = DirectoryLoader('PDF_Testing', glob="./*.pdf", loader_cls=PyPDFLoader)
loader = PyPDFLoader('How AI is expanding art history.pdf')
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

In [4]:
len(texts)

86

In [5]:
texts[5]

Document(page_content='perspective. Programs are recovering the \nappearance of lost or hidden artworks and \neven computing the ‘meanings’ of some paint -\nings, by identifying symbols, for example.', metadata={'source': 'How AI is expanding art history.pdf', 'page': 0})

In [6]:
_ = load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

llm=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta", 
    model_kwargs={"temperature":0.2, "max_length":256},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
    )

In [7]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="BAAI/bge-base-en-v1.5",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
%%time
persist_directory = 'db_HuggingFace'

embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: total: 1.3 s
Wall time: 3.31 s


In [9]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
docs = retriever.get_relevant_documents("What is paranoia?")

In [10]:
len(docs)

2

In [11]:
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [12]:
def process_llm_response(qa_chain, query):
    print(f"Query: {query}\n")
    time_1 = time()
    llm_response = qa_chain(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print(f"\nResult:", llm_response['result'])
    print(f"\nmetadata:", llm_response['source_documents'][0].metadata)

In [13]:
query = "How does AI affect Art?"
process_llm_response(qa_chain, query)

Query: How does AI affect Art?

Inference time: 3.794 sec.

Result:  In the study of art, AI is revealing new insights into color and style by analyzing large datasets of artworks. AI-driven tools are helping artists understand the science behind their creative choices, allowing them to make more informed decisions in their work. However, to fully harness the power of AI in art, we will need the same foundations as other fields, such as data privacy and transparency in algorithmic decision-making. Overall, AI is enhancing the creative process in art by providing new perspectives and tools for exploration.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [14]:
# break it down
query = "How does AI mimic human artwork?"
process_llm_response(qa_chain, query)

Query: How does AI mimic human artwork?

Inference time: 0.62 sec.

Result:  AI-driven tools are revealing how artists' understanding of the science of color and style contributes to their unique artistic style, allowing for the creation of AI-generated artwork that mimics the style of human artists. This is made possible by the same foundations required for the full power of AI in the study of art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [15]:
query = "How do they measure Momentary social anxiety?"
process_llm_response(qa_chain, query)

Query: How do they measure Momentary social anxiety?

Inference time: 0.582 sec.

Result:  The study assessed momentary social anxiety using three specific items suggested in a previous study. Unfortunately, the specific items are not provided in this context, so the exact measurement method is not clear. However, it can be inferred that these items likely focus on measuring the intensity and duration of social anxiety in specific situations or interactions. Without further information, it is impossible to provide a more detailed answer.

metadata: {'page': 5, 'source': 'PDF_file\\s41598-023-47912-0.pdf'}


In [16]:
query = "Please give me an example of using AI to produce an artwork"
process_llm_response(qa_chain, query)

Query: Please give me an example of using AI to produce an artwork

Inference time: 13.603 sec.

Result:  One example of using AI to produce an artwork is the "Portrait of Edmond de Belamy" created by a team of researchers at the French company, OpenAI. The portrait is a result of training a generative adversarial network (GAN) on a dataset of portraits from the 14th to the 20th century. The GAN consists of two neural networks, a generator that creates new images, and a discriminator that determines whether an image is real or fake. The generator learns to create more realistic images by trying to fool the discriminator, while the discriminator becomes better at distinguishing real images from the fake ones generated by the generator. The final result is a highly detailed and realistic portrait that is entirely generated by the AI system. This technology has the potential to revolutionize the art world by enabling the creation of entirely new forms of art that were previously impossibl

In [17]:
query = "How does machine learning and deep neural network help developing AI in art field?"
process_llm_response(qa_chain, query)

Query: How does machine learning and deep neural network help developing AI in art field?

Inference time: 11.562 sec.

Result:  Machine learning and deep neural networks are essential foundations for realizing the full power of AI in the study of art. These technologies allow AI-driven tools to analyze brush strokes, color, and style in fine-art paintings and drawings, revealing new insights and shedding light on these works of art. By providing the same foundations as other fields, machine learning and deep neural networks are helping to develop AI in the art field, unlocking new possibilities for understanding and appreciating art.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [18]:
query = "What is By-eye art analysis?"
process_llm_response(qa_chain, query)

Query: What is By-eye art analysis?



ValueError: Error raised by inference API: Internal Server Error

In [19]:
query = "Other than produce Artwork, what else can AI use in Art study?"
process_llm_response(qa_chain, query)

Query: Other than produce Artwork, what else can AI use in Art study?

Inference time: 13.658 sec.

Result:  AI can be used to analyze and interpret art, as well as to assist in conservation efforts by identifying areas of damage or deterioration in paintings and sculptures. It can also be used to help curate exhibitions by suggesting connections between different works of art based on their stylistic features. Additionally, AI can be used to assist in art education by providing personalized feedback to students on their artistic abilities and suggesting areas for improvement.

metadata: {'page': 2, 'source': 'How AI is expanding art history.pdf'}


In [20]:
query = "Give me an example of AI help recovering old art pieces"
process_llm_response(qa_chain, query)

Query: Give me an example of AI help recovering old art pieces

Inference time: 18.944 sec.

Result:  One way that AI can assist in recovering old art pieces is through the process of image inpainting. This technique uses machine learning algorithms to fill in missing or damaged areas of an image with plausible content based on the surrounding context. By applying image inpainting to old artworks with missing or damaged sections, art scholars can restore the original image to its former glory, providing a more complete and accurate representation of the artwork. This technology can also be used to help conserve and preserve delicate art pieces by reducing the need for physical restoration, which can sometimes cause further damage to the artwork. Overall, AI has the potential to revolutionize the way we study and appreciate old art pieces, helping us to better understand and appreciate the rich history of art and culture.

metadata: {'page': 2, 'source': 'How AI is expanding art history

In [ ]:
query = "What is the future of AI in art study?"
process_llm_response(qa_chain, query)

Query: What is the future of AI in art study?



In [ ]:
query = "Is there any challenge that AI facing in art study?"
process_llm_response(qa_chain, query)

In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)